In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import cv2
from tqdm.notebook import trange, tqdm

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
input_files = r'C:\Users\svo0175\Documents\Work\Svetlomet\Hella_06_05_2021\Hrabova_test_long_nove_svetlo\Hrabova_test_long_nove_svetlo_cut_{0}.{1}'
input_files = r'C:\Users\Petr\Documents\svetlomety\data\Hrabova_test_long_nove_svetlo\Hrabova_test_long_nove_svetlo_cut_{0}.{1}'

thresholds_arr = [10, 50, 100, 115]

In [ ]:
dfs = {}
for k in thresholds_arr:
    dfs[k] = pd.read_parquet(input_files.format(k, 'parquet.gzip'), engine='pyarrow')

In [ ]:
df = dfs[50]

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df.iloc[:100, :].apply(lambda x: x < 1.0, axis=1)

In [ ]:
df.describe()

In [ ]:
df_avg = pd.DataFrame()
df_avg['FrameTimestamp_us'] = df.FrameTimestamp_us
df_avg['avg_edge_occurence'] = df.iloc[:, :-1].mean(axis=1)
print(df_avg.shape)
df_avg.head()

In [ ]:
df_avg.describe()

In [ ]:
df_tmp = df_avg.iloc[:50000]
px.line(x=df_tmp.FrameTimestamp_us, y=df_tmp.avg_edge_occurence)

In [ ]:
df_tmp = df_avg.iloc[50000:150000]
px.line(x=df_tmp.FrameTimestamp_us, y=df_tmp.avg_edge_occurence)

In [ ]:
df_tmp = df_avg
px.line(x=df_tmp.FrameTimestamp_us, y=df_tmp.avg_edge_occurence)

In [ ]:
df_tmp = df_avg.iloc[50000:250000]
px.line(x=df_tmp.index, y=df_tmp.avg_edge_occurence)

In [ ]:
df_tmp = df_avg.iloc[10000:150000]
px.line(x=df_tmp.FrameTimestamp_us, y=df_tmp.avg_edge_occurence)

10 period v ruznych casovych usecich:

55.5241M - 57.88037M
131.3595M - 131.839M
239.5904M - 239.8361M
326.6124M - 326.7854M

### Kontrola velkych zmen - podezrele mista v kvalite video zaznamu

In [ ]:
(df_avg - df_avg.shift())[['FrameTimestamp_us', 'avg_edge_occurence']].describe(percentiles=[.01, .1, .25, .5, .75, .9, .99])

In [ ]:
df_avg['edge_shift'] = df_avg.avg_edge_occurence - df_avg.shift().avg_edge_occurence
px.line(x=df_avg.FrameTimestamp_us, y=df_avg.edge_shift)

In [ ]:
#df_avg['rolling_avg_edge_occurence'] = df_avg.rolling(800, win_type='triang').avg_edge_occurence.mean()
df_avg['smoothed_avg_edge_occurence'] = df_avg.rolling(10, center=True).avg_edge_occurence.mean()
df_avg['rolling_avg_edge_occurence'] = df_avg.rolling(800, center=True).avg_edge_occurence.mean()

In [ ]:
df_tmp = df_avg.iloc[10000:150000]
df_tmp = pd.melt(df_tmp, id_vars=['FrameTimestamp_us'], value_vars=['avg_edge_occurence', 'smoothed_avg_edge_occurence', 'rolling_avg_edge_occurence'])
px.line(df_tmp, x='FrameTimestamp_us', y='value', color='variable')

In [ ]:
df_avg.loc[df_avg.avg_edge_occurence>=df_avg.rolling_avg_edge_occurence, 'polarity'] = 1
df_avg.loc[df_avg.avg_edge_occurence<df_avg.rolling_avg_edge_occurence, 'polarity'] = -1
df_avg['polarity_shift_abs'] = abs(df_avg.polarity-df_avg.polarity.shift())
df_avg['polarity_shift'] = df_avg.polarity-df_avg.polarity.shift()
df_avg.loc[df_avg.smoothed_avg_edge_occurence>=df_avg.rolling_avg_edge_occurence, 'polarity_smoothed'] = 1
df_avg.loc[df_avg.smoothed_avg_edge_occurence<df_avg.rolling_avg_edge_occurence, 'polarity_smoothed'] = -1
df_avg['polarity_smoothed_shift_abs'] = abs(df_avg.polarity_smoothed-df_avg.polarity_smoothed.shift())
df_avg['polarity_smoothed_shift'] = df_avg.polarity_smoothed-df_avg.polarity_smoothed.shift()

In [ ]:
df_tmp = df_avg.iloc[10000:150000]
df_tmp = pd.melt(df_tmp, id_vars=['FrameTimestamp_us'], value_vars=['avg_edge_occurence', 'rolling_avg_edge_occurence', 'polarity', 'polarity_shift', 'polarity_smoothed', 'polarity_smoothed_shift'])
px.line(df_tmp, x='FrameTimestamp_us', y='value', color='variable')

In [ ]:
df_tmp = df_avg[(df_avg.FrameTimestamp_us >= 244925000) & (df_avg.FrameTimestamp_us <= 245200000)]
df_tmp = pd.melt(df_tmp, id_vars=['FrameTimestamp_us'], value_vars=['avg_edge_occurence', 'rolling_avg_edge_occurence', 'polarity', 'polarity_shift', 'polarity_smoothed', 'polarity_smoothed_shift'])
px.line(df_tmp, x='FrameTimestamp_us', y='value', color='variable')

In [ ]:
df_tmp = df_avg.iloc[10000:150000]
df_tmp = df_tmp.reset_index()
df_tmp = pd.melt(df_tmp, id_vars=['index'], value_vars=['avg_edge_occurence', 'rolling_avg_edge_occurence', 'polarity', 'polarity_shift_abs'])
px.line(df_tmp, x='index', y='value', color='variable')

In [ ]:
df_avg['rolling_q25'] = df_avg.rolling(800).avg_edge_occurence.quantile(.25)
df_avg['rolling_q75'] = df_avg.rolling(800).avg_edge_occurence.quantile(.75)

In [ ]:
df_tmp = df_avg.iloc[10000:150000]
df_tmp = pd.melt(df_tmp, id_vars=['FrameTimestamp_us'], value_vars=['avg_edge_occurence', 'rolling_avg_edge_occurence', 'rolling_q25', 'rolling_q75'])
px.line(df_tmp, x='FrameTimestamp_us', y='value', color='variable')